In [3]:
import os
import math
from PIL import Image ,  ImageOps
from imutils import paths
import numpy as np
import argparse
import imutils
import cv2
import glob
import matplotlib.pyplot as plt
import tensorflow as tf
import matplotlib.cm as cm

C:\Users\82106\miniconda3\envs\tf24\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


- 이미지 이어붙이기

In [4]:
def resize(filename):
    img = cv2.imread(filename)
    width, height = img.shape[:2]
    if height * width * 3 <= 2 ** 25:
        return img
    i = 2
    t_height, t_width = height, width
    while t_height * t_width * 3 > 2 ** 25:
        t_height = int(t_height / math.sqrt(i))
        t_width = int(t_width / math.sqrt(i))
        i += 1
    height, width = t_height, t_width
    image = Image.open(filename)
    resize_image = image.resize((height, width))
    filename = filename[:-1 * (len(filename.split(".")[-1]) + 1)] + "_resized." + filename.split(".")[-1]
    resize_image.save(filename)
    img = cv2.imread(filename)
    os.system("del " + filename.replace("/", "\\"))
    return img

In [10]:
# import argparse

# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--images", type=str, required=True,
#                 help="path to input directory of images to stitch")
# ap.add_argument("-o", "--output", type=str, required=True,
#                 help="path to the output image")

import easydict

args = easydict.EasyDict({ "images": 'images/', "ouput": 'output/'})
#args = vars(ap.parse_args())

In [11]:
# 이미지 불러오기
print("[INFO] loading images...")
imagePaths = sorted(list(paths.list_images(args["images"])))
images = []

for imagePath in imagePaths:
    image = resize(imagePath)
    images.append(image)

[INFO] loading images...


In [12]:
# 이어붙이기
stitcher = cv2.createStitcher() if imutils.is_cv3() else cv2.Stitcher_create()
(status, stitched) = stitcher.stitch(images)

In [13]:
# cv2.imshow("Stitched", stitched)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [14]:
# 이미지 저장, 출력
if status == 0:
    # write the output stitched image to disk
    cv2.imwrite('output/Stitched_02.png', stitched)

    # display the output stitched image to our screen
    cv2.imshow("Stitched", stitched)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    if status == cv2.STITCHER_ERR_NEED_MORE_IMGS:
        print("[INFO] image stitching failed (1: STITCHER_ERR_NEED_MORE_IMGS)")
    elif status == cv2.STITCHER_ERR_HOMOGRAPHY_EST_FAIL:
        print("[INFO] image stitching failed (2: STITCHER_ERR_HOMOGRAPHY_EST_FAIL)")
    else:
        print("[INFO] image stitching failed (3: STITCHER_ERR_CAMERA_PARAMETERS_ADJUSTMENT_FAIL)")

[INFO] image stitching failed (3: STITCHER_ERR_CAMERA_PARAMETERS_ADJUSTMENT_FAIL)


- 좌표 출력

In [ ]:
# 이진화(예측) 이미지 불러오기
map_img = cv2.imread('output/Stitched_02.png', 0)

In [ ]:
map_img = cv2.resize(map_img, dsize = (256,256))

In [ ]:
width, height = map_img.shape
map = np.zeros((width, height), np.uint8)

In [ ]:
for x in range(height):
    for y in range(width):
        if map_img[x][y] == 255:
            map[x][y] = 1
        else:
            map[x][y] = 0

In [ ]:
for x in range(height):
    for y in range(width):
        if map[x][y] == 1:
            print(x, y)
            
np.count_nonzero(map)